In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException, TimeoutException
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from IPython.display import display
from csv import writer
import csv
import time
import warnings
warnings.filterwarnings('ignore')
import pandas as pd

In [2]:
def get_jobs(num_jobs, verbose):
    
    s = Service('D:\Selenium driver\chromedriver2.exe')
    driver = webdriver.Chrome(service=s)
    driver.maximize_window()   
    url = 'https://www.glassdoor.com/Job/canada-business-analyst-jobs-SRCH_IL.0,6_IN3_KO7,23_IP20.htm'
    driver.get(url)
    time.sleep(2)
    jobs = []
    while len(jobs) < num_jobs:
        
        time.sleep(4)
        
        job_buttons = driver.find_elements_by_xpath('//li[@data-ad-order-id="1136043"]')
       
        for job_button in job_buttons:
            
            print("Progress: {}".format("" + str(len(jobs)) + "/" + str(num_jobs)))
            if len(jobs) >= num_jobs:
                break

            job_button.click() 
            time.sleep(1)
            try:
                try:
                    driver.find_element_by_xpath('//span[@alt="Close"]').click()
                except NoSuchElementException:
                    pass  
                time.sleep(1)
                collected_successfully = False

                while not collected_successfully:
                    try:
                        company_name = driver.find_element_by_xpath('.//div[@class="css-xuk5ye e1tk4kwz5"]').text
                        location = driver.find_element_by_xpath('.//div[@class="css-56kyx5 e1tk4kwz1"]').text
                        job_title = driver.find_element_by_xpath('.//div[@class="css-1j389vi e1tk4kwz2"]').text

                        collected_successfully = True
                    except:
                        time.sleep(5)

                    try:
                        job_description = driver.find_element_by_xpath('.//div[@class="jobDescriptionContent desc"]').get_attribute('textContent')
                    except NoSuchElementException:
                        job_description = -1


                try:
                    salary_estimate = job_button.find_element_by_xpath('.//div[@class="css-1buaf54 pr-xxsm"]/span').text
                except NoSuchElementException:
                    salary_estimate = -1 #You need to set a "not found value. It's important."

                try:
                    rating = driver.find_element_by_xpath('.//span[@data-test="detailRating"]').text
                except NoSuchElementException:
                    rating = -1 #You need to set a "not found value. It's important."

                #Printing for debugging
                if verbose:
                    print("Job Title: {}".format(job_title))
                    print("Job Description: {}".format(job_description))
                    print("Salary Estimate: {}".format(salary_estimate))
                    print("Rating: {}".format(rating))
                    print("Company Name: {}".format(company_name))
                    print("Location: {}".format(location))
                time.sleep(2)

                try:
                    driver.find_element_by_xpath('.//div[@class="css-1ap6ha9 ef7s0la0"]').click()

                    try:
                        headquarters = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Headquarters"]//following-sibling::*').text
                    except NoSuchElementException:
                        headquarters = -1

                    try:
                        size = driver.find_element_by_xpath('.//div[@class="d-flex justify-content-start css-daag8o e1pvx6aw2"]//span[text()="Size"]//following-sibling::*').text
                    except NoSuchElementException:
                        size = -1

                    try:
                        founded = driver.find_element_by_xpath('.//div[@class="d-flex justify-content-start css-daag8o e1pvx6aw2"]//span[text()="Founded"]//following-sibling::*').text
                    except NoSuchElementException:
                        founded = -1

                    try:
                        type_of_ownership = driver.find_element_by_xpath('.//div[@class="d-flex justify-content-start css-daag8o e1pvx6aw2"]//span[text()="Type"]//following-sibling::*').text
                    except NoSuchElementException:
                        type_of_ownership = -1

                    try:
                        industry = driver.find_element_by_xpath('.//div[@class="d-flex justify-content-start css-daag8o e1pvx6aw2"]//span[text()="Industry"]//following-sibling::*').text
                    except NoSuchElementException:
                        industry = -1

                    try:
                        sector = driver.find_element_by_xpath('.//div[@class="d-flex justify-content-start css-daag8o e1pvx6aw2"]//span[text()="Sector"]//following-sibling::*').text
                    except NoSuchElementException:
                        sector = -1

                    try:
                        revenue = driver.find_element_by_xpath('.//div[@class="d-flex justify-content-start css-daag8o e1pvx6aw2"]//span[text()="Revenue"]//following-sibling::*').text
                    except NoSuchElementException:
                        revenue = -1

                    try:
                        competitors = driver.find_element_by_xpath('.//div[@class="d-flex justify-content-start css-daag8o e1pvx6aw2"]//span[text()="Competitors"]//following-sibling::*').text
                    except NoSuchElementException:
                        competitors = -1

                except NoSuchElementException:  #Rarely, some job postings do not have the "Company" tab.
                    headquarters = -1
                    size = -1
                    founded = -1
                    type_of_ownership = -1
                    industry = -1
                    sector = -1
                    revenue = -1
                    competitors = -1

                if verbose:
                    print("Headquarters: {}".format(headquarters))
                    print("Size: {}".format(size))
                    print("Founded: {}".format(founded))
                    print("Type of Ownership: {}".format(type_of_ownership))
                    print("Industry: {}".format(industry))
                    print("Sector: {}".format(sector))
                    print("Revenue: {}".format(revenue))
                    print("Competitors: {}".format(competitors))
                    print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@")
                #add job to jobs
                jobs.append({"Job Title" : job_title,
                "Salary Estimate" : salary_estimate,
                "Job Description" : job_description,
                "Rating" : rating,
                "Company Name" : company_name,
                "Location" : location,
                "Headquarters" : headquarters,
                "Size" : size,
                "Founded" : founded,
                "Type of ownership" : type_of_ownership,
                "Industry" : industry,
                "Sector" : sector,
                "Revenue" : revenue,
                "Competitors" : competitors})
            except:
                job_button.click()
        #Clicking on the "next page" button
        try:
            driver.find_element_by_xpath('//button[@aria-label="Next"]').click()
            time.sleep(3)
        except NoSuchElementException:
            print("Scraping terminated before reaching target number of jobs. Needed {}, got {}.".format(num_jobs, len(jobs)))
            pass

    return pd.DataFrame(jobs)  #This line converts the dictionary object into a pandas DataFrame.


In [ ]:
df = get_jobs(500, False)
df.to_csv("BA Salary 2.csv")

Progress: 0/500
